In [1]:
import os, sys
import tensorflow as tf
import pandas as pd 
import numpy as np 
import nltk
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from autocorrect import Speller
from tensorflow.keras.layers import TextVectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
nltk.download('stopwords')
nltk.download('wordnet')
# Download NLTK resources
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
print(tf.__version__)

2.13.0


In [3]:
train_df = pd.read_csv(r'C:/Users/nikhi/Comment_toxicity/dataset/train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test_df = pd.read_csv(r'C:/Users/nikhi/Comment_toxicity/dataset/test.csv')
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            153164 non-null  object
 1   comment_text  153164 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [7]:
train_df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [8]:
test_df.isnull().sum()

id              0
comment_text    0
dtype: int64

In [9]:
contraction_dict = {"arent't":"are not", "s":"is", "ain't":"are not","don't":"do not"}
contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))

In [10]:
def expand_contractions(text, contraction_dict=contraction_dict):
    def replace(match):
        return contraction_dict[match.group(0)]
    return contraction_re.sub(replace, text)

df = train_df.copy()
df['comment_text'] = df['comment_text'].apply(lambda x: expand_contractions(x))


In [11]:
df['comment_text']

0         Explanation\nWhy the editis made under my uise...
1         D'aww! He matcheis thiis background colour I'm...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real isuggeistionis ...
4         You, isir, are my hero. Any chance you remembe...
                                ...                        
159566    ":::::And for the isecond time of aisking, whe...
159567    You ishould be aishamed of youriself \n\nThat ...
159568    Spitzer \n\nUmm, thereis no actual article for...
159569    And it lookis like it wais actually you who pu...
159570    "\nAnd ... I really do not think you underista...
Name: comment_text, Length: 159571, dtype: object

In [12]:
def clean_text(text):
    # Remove newline characters
    text = re.sub('\n', ' ', text)
    # Remove other unwanted characters or symbols
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespaces
    text = re.sub('\s+', ' ', text).strip()
    
    return text.lower()
    
    

df['comment_text'] = df['comment_text'].apply(lambda x:clean_text(x))

In [13]:
# def corrected_spelling(text):
#     spell = Speller(lang='en')
#     tokens = word_tokenize(text)
#     corrected_tokens = [spell(word) for word in tokens if word is not None]
#     corrected_text = " ".join(corrected_tokens)
#     return corrected_text

# df['comment_text'] = df['comment_text'].apply(lambda x: corrected_spelling(x))

In [14]:
# def corrected_spelling(text):
#     if isinstance(text, str):
#         spell = SpellChecker()
#         tokens = nltk.word_tokenize(text)
#         corrected_tokens = [spell.correction(word) for word in tokens if word is not None]
#         corrected_text = " ".join(corrected_tokens)
#         return corrected_text
#     else:
#         return ''


# df['comment_text'] = df['comment_text'].apply(corrected_spelling)

In [15]:
df['comment_text']

0         explanation why the editis made under my uiser...
1         daww he matcheis thiis background colour im is...
2         hey man im really not trying to edit war itis ...
3         more i cant make any real isuggeistionis on im...
4         you isir are my hero any chance you remember w...
                                ...                        
159566    and for the isecond time of aisking when your ...
159567    you ishould be aishamed of youriself that iis ...
159568    spitzer umm thereis no actual article for proi...
159569    and it lookis like it wais actually you who pu...
159570    and i really do not think you underistand i ca...
Name: comment_text, Length: 159571, dtype: object

In [16]:
def preprocess_text(text):
    if isinstance(text, str):
    
        tokens = nltk.word_tokenize(text)
        # remove stopwords
        stop_words = set(stopwords.words('english'))
        filtered_text = [word for word in tokens if word.lower() not in stop_words]

        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_text]

        preprocessed_text = " ".join(lemmatized_tokens)
        return preprocessed_text 
    else:
        return '' 


In [17]:
df['comment_text'] = df['comment_text'].apply(lambda x:preprocess_text(x))

In [18]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation editis made uisername hardcore met...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matcheis thiis background colour im iseem...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really trying edit war itis juist t...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real isuggeistionis improvement wond...,0,0,0,0,0,0
4,0001d958c54c6e35,isir hero chance remember page thatis,0,0,0,0,0,0


In [19]:
df.drop('id', axis=1, inplace=True)

In [20]:
df[df.columns[1:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [21]:
X = df['comment_text']
y = df[df.columns[1:]].values

In [26]:
MAX_FEATURES = 200000
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=1800,
    output_mode='int'
)

In [27]:
vectorizer.adapt(X.values)

In [28]:
len(vectorizer.get_vocabulary())

200000

In [29]:
vectorized_text = vectorizer(X.values)

In [30]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [31]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [32]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [33]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [34]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 5451s 780ms/step - loss: 0.0621 - val_loss: 0.0427
